In [1]:
import os
from openai import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
import torch

In [2]:
# print(os.getenv("OPENAI_API_KEY"))

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Audio Extractor

In [4]:
def audio_extractor(file_path):
    with open(file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    return transcript.text

In [5]:
# print(audio_extractor("Artificia_lNeurons_for_Music_and_Sound_Design_ Simon_Hutchinson.mp3"))

In [6]:
# Quantization using BitsAndBytes 
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [7]:
model_id = "d:\\Models\\Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
transcript = audio_extractor("Artificia_lNeurons_for_Music_and_Sound_Design_ Simon_Hutchinson.mp3")
messages = [
    {"role": "system", "content": "You are a helpful assistant that summarizes meeting text into concise minutes."},
    {"role": "user", "content": f"Please generate the minutes from the following transcript:\n {transcript}"}
]

# format the message for mistral ai
def format_chat(messages):
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

input_text = format_chat(messages)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate, Decode and Print
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)

# Removing Prompt manually from the output text
input_len = inputs.input_ids.shape[1]  # Length of the prompt
generated_text = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Title: Exploration of Artificial Neurons in Sound Synthesis by Simon Hutchinson

1. Introduction
   - Simon Hutchinson, a composer, music technologist, and professor of music at the University of New Haven, discussed his work with artificial neurons and their implementation in sound synthesis.
   - He expressed excitement about this work due to its connections to feedback, audio non-linearity, and machine learning.

2. Focus on Artificial Neurons
   - Artificial neurons are mathematical functions that mimic biological neurons, serving as the building blocks of artificial neural networks.
   - They can be created through code or electronics and function as a mixer with a non-linear transformation.

3. Non-Linearity in Artificial Neurons
   - The non-linearity in artificial neurons makes the behavior of the network more complex, preventing simplification to a single neuron.
   - Examples of non-linear activation functions include the hyperbolic tangent function and rectified linear unit.